In [1]:
import pandas as pd
import altair as alt
from altair import datum
import numpy as np
import os

In [2]:
alt.renderers.enable('default')

userhome = os.path.expanduser('~')
csvfile = userhome + r'/Downloads/WW1_BOMBING_DATA-CLEANED.csv'

# Use 'ISO-8859-1' encoding to read the CSV file
df = pd.read_csv(csvfile, parse_dates=['MSNDATE'], encoding='ISO-8859-1', dayfirst=True)

df_filtered = df.dropna(subset=['TGTCOUNTRY', 'COUNTRY', 'MSNDATE']).copy()

df_filtered.head()

,WWI_ID,MSNDATE,OPERATION,COUNTRY,SERVICE,UNIT,MDS,MISSIONNUM,TAKEOFFTIME,NUMBEROFPLANESATTACKING,...,TAKEOFFBASE,TAKEOFFLATITUDE,TAKEOFFLONGITUDE,BDA,ENEMYACTION,ROUTEDETAILS,ISRCOLLECTED,FRIENDLYCASUALTIES,WEATHER,ALTITUDE
0,1,1917-10-16,NaN,UK,RAF,NO. 55 SQDN,AIRCO DH4 DAY BOMBER,NaN,DAY,8,...,OCHEY,48.581292,5.953517,NaN,NaN,NaN,NaN,0,NaN,10000.0
1,2,1917-10-20,NaN,UK,RAF,NO. 55 SQDN,AIRCO DH4 DAY BOMBER,NaN,DAY,11,...,OCHEY,48.581292,5.953517,NaN,NaN,NaN,NaN,1,NaN,10000.0
2,3,1917-10-23,NaN,UK,RAF,NO. 100 SQDN,FE2B,NaN,NIGHT,12,...,OCHEY,48.581292,5.953517,NaN,NaN,NaN,NaN,2,NaN,10000.0
3,4,1917-10-23,NaN,UK,RAF,NO. 216 SQDN,HADLEY PAGE O/100,NaN,NIGHT,7,...,VILLESENEUX,48.781080,4.163132,NaN,NaN,NaN,NaN,2,NaN,10000.0
4,5,1917-10-28,NaN,UK,RAF,NO. 100 SQDN,FE2B,NaN,NIGHT,3,...,OCHEY,48.581292,5.953517,NaN,NaN,NaN,NaN,0,NaN,10000.0


In [3]:

df_filtered.loc[:, 'MSNDATE'] = pd.to_datetime(df_filtered['MSNDATE'])
# determine opacity
slider = alt.binding_range(min=0, max=1, step=0.05, name='opacity:')
op_var = alt.param(value=0.7, bind=slider)

# interval selection for the brushing
brush = alt.selection_interval()
single = alt.selection_point()

# Add new 'month' and 'year' columns to the DataFrame
df_filtered.loc[:, 'month'] = pd.to_datetime(df_filtered['MSNDATE'], format='%M').dt.month.astype(int)
df_filtered.loc[:, 'year'] = pd.to_datetime(df_filtered['MSNDATE'], format='%Y').dt.year.astype(int)

# selection for the date
date_selection = alt.selection_point(fields=['MSNDATE'], nearest=True, on='click')
year_selection = alt.selection_point(fields=['year'], nearest=True, on='click')
month_selection = alt.selection_point(fields=['month'], nearest=True, on='click')

mission_selection = alt.selection_point(fields=['WWI_ID'])

# selection for the legend
selection = alt.selection_point(fields=['TGTCOUNTRY'])

# selection for the country
country_selection = alt.selection_point(fields=['COUNTRY'], nearest=True, on='click')

# ombined list of unique countries from both 'COUNTRY' and 'TGTCOUNTRY'
unique_countries = pd.concat([df_filtered['COUNTRY'], df_filtered['TGTCOUNTRY']]).unique()

# color scheme that maps each unique country to a color
color_scheme = alt.Scale(domain=list(unique_countries), scheme='category20')

# color condition for 'COUNTRY'
attackercolor = alt.condition(
    selection & brush,
    alt.Color('COUNTRY:N', scale=color_scheme).legend(None),
    alt.value('lightgray')
)

# Ccolor condition for 'TGTCOUNTRY'
color = alt.condition(
    selection & brush,
    alt.Color('TGTCOUNTRY:N', scale=color_scheme).legend(None),
    alt.value('lightgray')
)

bombing_over_time_color = alt.condition(
    selection,
    alt.Color('COUNTRY:N', scale=color_scheme).legend(None),
    alt.value('lightgray')
)

# to show the legend
legend = alt.Chart(df_filtered).mark_bar(opacity=op_var).encode(
    alt.Y('TGTCOUNTRY').axis(orient='left', title='Targeted country'),
    color = color
).add_params(op_var, selection)

# to show the attacking countries by targeted countries
attacked_by = alt.Chart(df_filtered).mark_bar(opacity=op_var).encode(
    alt.X('COUNTRY', axis=alt.Axis(title='Attacking country')),
    alt.Y('TGTCOUNTRY', axis=alt.Axis(title='Targeted country')),
    color = attackercolor,
    tooltip = ["COUNTRY", "TGTCOUNTRY", 'count(TGTCOUNTRY):Q']
).add_params(op_var, brush, country_selection, selection).interactive()

# to show the proportion of attacking countries by targeted countries
bombed_count = alt.Chart(df_filtered).mark_bar(opacity=op_var).encode(
    alt.X('count(COUNTRY):Q', stack="normalize", axis=alt.Axis(title='Proportion of Attacking countries')),
    alt.Y('TGTCOUNTRY:N', axis=alt.Axis(title='Targeted country')),
    color = attackercolor,
    tooltip = ["COUNTRY", 'TGTCOUNTRY', 'count(COUNTRY):Q']
).add_params(op_var, brush, selection)

# to show the number of being bombed by targeted countries on a timeline
bombing_over_time = alt.Chart(df_filtered, title='Number of Bombings by Targeted Countries Over Time').mark_circle(opacity=op_var).encode(
    alt.X('MSNDATE:T', axis=alt.Axis(title='Date'), scale=alt.Scale(domain=['1915', '1919'], clamp=True)),
    alt.Y('COUNTRY', axis=alt.Axis(title='Attacking country'), scale=alt.Scale(nice=10, zero=False, padding=1)),  
    color = bombing_over_time_color,
    order = alt.Order('count(TGTCOUNTRY):Q', sort='descending'),
    size = alt.Size('count(TGTCOUNTRY):Q', 
                    title='Number of being bombed', 
                    scale=alt.Scale(range=[0, 1000]),
                    legend=alt.Legend(orient='top', titleOrient='left')),
    tooltip = ["COUNTRY", "TGTCOUNTRY", 'count(TGTCOUNTRY):Q', 'MSNDATE:T']
).transform_filter(
    selection&brush
).add_params(
    op_var, date_selection, month_selection, year_selection
).interactive().properties(width=650)

# For the same day
same_day_incidents = alt.Chart(df_filtered, title='Incidents in the same day').mark_circle(opacity=op_var).encode(
    alt.X('WEAPONWEIGHT', axis=alt.Axis(title='Weapon weight')),
    alt.Y('BOMBLOAD', axis=alt.Axis(title='Bombload'), scale=alt.Scale(nice=10, zero=False, padding=1)),  
    color = attackercolor,
    size = alt.value(150),
    tooltip = ["COUNTRY", "TGTCOUNTRY", 'MSNDATE:T', 'WEAPONWEIGHT', 'BOMBLOAD']
).transform_filter(
    date_selection
).add_params(
    op_var, brush, date_selection
).interactive().properties(height=200, width=650)

In [4]:
# For the same year
same_year_incidents = alt.Chart(df_filtered, title='Incidents in the same year').mark_circle(opacity=op_var).encode(
    alt.X('WEAPONWEIGHT', axis=alt.Axis(title='Weapon weight')),
    alt.Y('BOMBLOAD', axis=alt.Axis(title='Bombload'), scale=alt.Scale(nice=10, zero=False, padding=1)),  
    color = attackercolor,
    size = alt.value(150),
    tooltip = ["COUNTRY", "TGTCOUNTRY", 'MSNDATE:T', 'WEAPONWEIGHT', 'BOMBLOAD']
).transform_filter(
    # (datum.year == alt.expr.year(date_selection.MSNDATE))
    # date_selection&year_selection
    year_selection
).add_params(
    op_var, brush
).interactive().properties(height=200, width=650)

# For the same month
same_month_incidents = alt.Chart(df_filtered, title='Incidents in the same month').mark_circle(opacity=op_var).encode(
    alt.X('WEAPONWEIGHT', axis=alt.Axis(title='Weapon weight')),
    alt.Y('BOMBLOAD', axis=alt.Axis(title='Bombload'), scale=alt.Scale(nice=10, zero=False, padding=1)),  
    color = attackercolor,
    size = alt.value(150),
    tooltip = ["COUNTRY", "TGTCOUNTRY", 'MSNDATE:T', 'WEAPONWEIGHT', 'BOMBLOAD']
).transform_filter(
    # (datum.year == alt.expr.year(date_selection.MSNDATE)) & 
    # (datum.month == alt.expr.month(date_selection.MSNDATE))
    month_selection
).add_params(
    op_var, brush
).interactive().properties(height=200, width=650)

In [5]:
year_selection.to_dict()

{'param': 'param_5'}

In [6]:
df_filtered.dtypes

WWI_ID                              int64
MSNDATE                    datetime64[ns]
OPERATION                          object
COUNTRY                            object
SERVICE                            object
UNIT                               object
MDS                                object
MISSIONNUM                        float64
TAKEOFFTIME                        object
NUMBEROFPLANESATTACKING            object
CALLSIGN                           object
WEAPONSEXPENDED                    object
WEAPONTYPE                         object
WEAPONWEIGHT                      float64
BOMBLOAD                          float64
LATITUDE                          float64
LONGITUDE                         float64
TGTLOCATION                        object
TGTCOUNTRY                         object
TGTTYPE                            object
TAKEOFFBASE                        object
TAKEOFFLATITUDE                   float64
TAKEOFFLONGITUDE                  float64
BDA                               

In [7]:
print(df_filtered.head())

   WWI_ID    MSNDATE OPERATION COUNTRY SERVICE          UNIT  \
0       1 1917-10-16       NaN      UK     RAF   NO. 55 SQDN   
1       2 1917-10-20       NaN      UK     RAF   NO. 55 SQDN   
2       3 1917-10-23       NaN      UK     RAF  NO. 100 SQDN   
3       4 1917-10-23       NaN      UK     RAF  NO. 216 SQDN   
4       5 1917-10-28       NaN      UK     RAF  NO. 100 SQDN   

                    MDS  MISSIONNUM TAKEOFFTIME NUMBEROFPLANESATTACKING  ...  \
0  AIRCO DH4 DAY BOMBER         NaN         DAY                       8  ...   
1  AIRCO DH4 DAY BOMBER         NaN         DAY                      11  ...   
2                  FE2B         NaN       NIGHT                      12  ...   
3     HADLEY PAGE O/100         NaN       NIGHT                       7  ...   
4                  FE2B         NaN       NIGHT                       3  ...   

  TAKEOFFLONGITUDE  BDA ENEMYACTION  ROUTEDETAILS  ISRCOLLECTED  \
0         5.953517  NaN         NaN           NaN           NaN   


In [8]:
print(bombing_over_time.to_json())

{
  "$schema": "https://vega.github.io/schema/vega-lite/v5.16.3.json",
  "config": {
    "view": {
      "continuousHeight": 300,
      "continuousWidth": 300
    }
  },
  "data": {
    "name": "data-7e6c590a08333399603a6b3cc9951a69"
  },
  "datasets": {
    "data-7e6c590a08333399603a6b3cc9951a69": [
      {
        "ALTITUDE": 10000.0,
        "BDA": null,
        "BOMBLOAD": 224.0,
        "CALLSIGN": null,
        "COUNTRY": "UK",
        "ENEMYACTION": null,
        "FRIENDLYCASUALTIES": "0",
        "ISRCOLLECTED": null,
        "LATITUDE": 49.233333,
        "LONGITUDE": 7.0,
        "MDS": "AIRCO DH4 DAY BOMBER",
        "MISSIONNUM": null,
        "MSNDATE": "1917-10-16T00:00:00",
        "NUMBEROFPLANESATTACKING": "8",
        "OPERATION": null,
        "ROUTEDETAILS": null,
        "SERVICE": "RAF",
        "TAKEOFFBASE": "OCHEY",
        "TAKEOFFLATITUDE": 48.58129185,
        "TAKEOFFLONGITUDE": 5.95351696,
        "TAKEOFFTIME": "DAY",
        "TGTCOUNTRY": "GERMANY",
    

In [9]:
((legend | attacked_by | bombed_count) & 
(bombing_over_time) &
(same_day_incidents) & 
(same_month_incidents) & 
(same_year_incidents)).save('WW1_BOMBING_SYS_TERRY.html')

In [10]:
print(df_filtered['MSNDATE'].dtypes)

datetime64[ns]


In [11]:
same_month_data = df_filtered[(df_filtered['MSNDATE'].dt.year == 1917) & (df_filtered['MSNDATE'].dt.month == 7)]
print(same_month_data)

     WWI_ID    MSNDATE OPERATION COUNTRY SERVICE UNIT            MDS  \
918     919 1917-07-15       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
919     920 1917-07-22       NaN   ITALY     NaN  NaN        AIRSHIP   
920     921 1917-07-26       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
922     923 1917-07-28       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
923     924 1917-07-30       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
966     967 1917-07-22       NaN   ITALY     NaN  NaN        AIRSHIP   
971     972 1917-07-08       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
972     973 1917-07-14       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
974     975 1917-07-16       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
975     976 1917-07-18       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
976     977 1917-07-25       NaN   ITALY     NaN  NaN        AIRSHIP   
979     980 1917-07-25       NaN   ITALY     NaN  NaN  CAPRONI CA.32   
983     984 1917-07-28       NaN   ITALY     NaN  NaN  CAPRONI C

In [12]:
df_filtered['SERVICE'].unique()

array(['RAF', 'USAAS', 'GAR', nan, 'ARMY', 'NAVY'], dtype=object)

In [13]:
((legend | attacked_by | bombed_count) & 
(bombing_over_time) &
(same_day_incidents) & 
(same_month_incidents) & 
(same_year_incidents))

alt.VConcatChart(...)